In [1]:
import psycopg2
import psycopg2.extras
import os
from dotenv import load_dotenv
import pandas as pd
import pandas_ta as ta
import numpy as np
import time
from IPython.display import display
import plotly.graph_objects as go

load_dotenv()

# Declare global variables
conn = None
cur = None

def get_db_connection():
    global conn, cur  # Declare globals to modify them inside the function

    if conn is None:
        conn = psycopg2.connect(
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            database="vcp",
        )
    
    if cur is None:
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    return conn, cur

def close_db_connection():
    global conn, cur  # Access globals to close them properly

    if cur:
        cur.close()
        cur = None  # Reset cur after closing
    
    if conn:
        conn.close()
        conn = None  # Reset conn after closing

def insert_by_token_(cur, instrument_token, interval):
        select_query = """SELECT * FROM historical_data WHERE instrument_token = %s AND interval = %s;"""
        cur.execute(select_query, (instrument_token, interval))
        return cur.fetchall()

conn, cur = get_db_connection()

In [2]:
import pandas as pd

# file_path = 'sml100.csv'
file_path = '500.csv'


df = pd.read_csv(file_path)

# Rename columns
df.columns = ['Company Name', 'Industry', 'tradingsymbol', 'Series', 'ISIN Code']
display(df)

# Fetch all instruments from the database
select_query = """SELECT * FROM equity_instruments;"""
cur.execute(select_query)

all_inst_df = pd.DataFrame(
    cur.fetchall(),
    columns=[
        'instrument_token', 'exchange_token', 'tradingsymbol', 'name',
        'last_price', 'tick_size', 'instrument_type', 'segment', 'exchange'
    ]
)
display(all_inst_df)

# Merge DataFrames
filtered_df = pd.merge(df, all_inst_df, on='tradingsymbol', how='left')
display(filtered_df)

# Create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS equity_tokens (
    instrument_token INTEGER PRIMARY KEY,
    tradingsymbol VARCHAR(255),
    company_name VARCHAR(255),
    exchange VARCHAR(255),
    CONSTRAINT unique_company_symbol UNIQUE (tradingsymbol, instrument_token)
);
"""
cur.execute(create_table_query)
conn.commit()

# Convert DataFrame columns to appropriate types
filtered_df = filtered_df.dropna(subset=['instrument_token', 'exchange'])  # Remove rows with missing values
filtered_df['instrument_token'] = filtered_df['instrument_token'].astype(int)
filtered_df['tradingsymbol'] = filtered_df['tradingsymbol'].astype(str)
filtered_df['Company Name'] = filtered_df['Company Name'].astype(str)
filtered_df['exchange'] = filtered_df['exchange'].astype(str)

# Batch insert into the database
insert_query = """
INSERT INTO equity_tokens (instrument_token, tradingsymbol, company_name, exchange)
VALUES (%s, %s, %s, %s)
ON CONFLICT (tradingsymbol, instrument_token) DO NOTHING;
"""

data_to_insert = []

# Iterate over the rows of the DataFrame
for _, row in filtered_df.iterrows():
    data_to_insert.append((
        row['instrument_token'],
        row['tradingsymbol'],
        row['Company Name'],
        row['exchange']
    ))

cur.executemany(insert_query, data_to_insert)
conn.commit()


,Company Name,Industry,tradingsymbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026
...,...,...,...,...,...
749,Wonderla Holidays Ltd.,Consumer Services,WONDERLA,EQ,INE066O01014
750,Yatharth Hospital & Trauma Care Services Ltd.,Healthcare,YATHARTH,EQ,INE0JO301016
751,Zen Technologies Ltd.,Capital Goods,ZENTEC,EQ,INE251B01027
752,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01010


,instrument_token,exchange_token,tradingsymbol,name,last_price,tick_size,instrument_type,segment,exchange
0,257,1,GOLDSTAR-SM,GOLDSTAR POWER,0.0,0.05,EQ,NSE,NSE
1,1793,7,AARTIIND,AARTI INDUSTRIES,0.0,0.05,EQ,NSE,NSE
2,2561,10,ABAN,ABAN OFFSHORE,0.0,0.01,EQ,NSE,NSE
3,3329,13,ABB,ABB INDIA,0.0,0.05,EQ,NSE,NSE
4,3585,14,656KA30-SG,SDL KA 6.56% 2030,0.0,0.01,EQ,NSE,NSE
...,...,...,...,...,...,...,...,...,...
7218,8150273,31837,SREEL,SREELEATHERS,0.0,0.05,EQ,NSE,NSE
7219,8183297,31966,863NHB29-N2,,0.0,0.01,EQ,NSE,NSE
7220,8183553,31967,863NHB34-N3,,0.0,0.01,EQ,NSE,NSE
7221,8184065,31969,888NHB29-N5,,0.0,0.01,EQ,NSE,NSE


,Company Name,Industry,tradingsymbol,Series,ISIN Code,instrument_token,exchange_token,name,last_price,tick_size,instrument_type,segment,exchange
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038,3343617.0,13061.0,360 ONE WAM,0.0,0.05,EQ,NSE,NSE
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017,121345.0,474.0,3M INDIA,0.0,0.05,EQ,NSE,NSE
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022,3329.0,13.0,ABB INDIA,0.0,0.05,EQ,NSE,NSE
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025,5633.0,22.0,ACC,0.0,0.05,EQ,NSE,NSE
4,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026,3350017.0,13086.0,AIA ENGINEERING,0.0,0.05,EQ,NSE,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,Wonderla Holidays Ltd.,Consumer Services,WONDERLA,EQ,INE066O01014,768513.0,3002.0,WONDERLA HOLIDAYS,0.0,0.05,EQ,NSE,NSE
750,Yatharth Hospital & Trauma Care Services Ltd.,Healthcare,YATHARTH,EQ,INE0JO301016,4540929.0,17738.0,YATHARTH HOSP & TRA C S L,0.0,0.05,EQ,NSE,NSE
751,Zen Technologies Ltd.,Capital Goods,ZENTEC,EQ,INE251B01027,1922049.0,7508.0,ZEN TECHNOLOGIES,0.0,0.05,EQ,NSE,NSE
752,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01010,4514561.0,17635.0,ZYDUS WELLNESS,0.0,0.05,EQ,NSE,NSE
